In [90]:
import pandas as pd

def load_and_clean_data(file_path):
    """
    Load and clean the data from the specified CSV file.
    
    Parameters:
    file_path (str): The path to the CSV file.
    
    Returns:
    pd.DataFrame: Cleaned DataFrame with renamed columns and date parsing.
    """
    # Load the data
    df = pd.read_csv(file_path)
    #alue = file_path.split('/')[-1].split('.')[0]
    # Rename columns for consistency
    #df.columns = ['observation_date', value]
    
    # Convert observation_date to datetime
    df['observation_date'] = pd.to_datetime(df['observation_date'])
    
    # Convert value to numeric, coercing errors to NaN
    #df[value] = pd.to_numeric(df[value], errors='coerce')
    
    return df

def merge_dataframes(df_list):
    """
    Merge a list of DataFrames on the observation_date column.
    
    Parameters:
    df_list (list): List of DataFrames to merge.
    
    Returns:
    pd.DataFrame: Merged DataFrame.
    """
    merged_df = df_list[0]
    for df in df_list[1:]:
        merged_df = pd.merge(merged_df, df, on='observation_date', how='outer')
    
    return merged_df


In [91]:
file_paths = [
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/FEDInterestRate.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/Construction_Wage.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/Construction_Employees.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/Construct_GDP_q_seasonal.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/CA_Unemployment.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/CA_investment_a.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/CPI.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/StateCorpTax_a.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/CA_energyprice_a.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/Mfg_Employees.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/Mfg_Wage.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/Min_Wage_a.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/ProducerPriceIndexUSA.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/avg_tarrif_a.csv', 
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/EnergyGen.csv',
        '/Users/gabrielketron/CAmfg/CAmfg/Data/Statewide-Monthly/Mfg_GDP_q_seasonal.csv'
    ]
    
# Load and clean each DataFrame
df_list = [load_and_clean_data(file_path) for file_path in file_paths]

# Merge all DataFrames
df = merge_dataframes(df_list)

# Display the final DataFrame
#print(df)
df = df.ffill()
df = df.dropna()
df = df[df['observation_date'] < '2024-01-01']
df['AdjustmentCPI'] = df['CPI'].iloc[127]/df['CPI']
df['SMU06000002000000003($ per hour)'] = df['SMU06000002000000003($ per hour)'] * df['AdjustmentCPI']
df['CACORPINCTX($)'] = df['CACORPINCTX($)'] * df['AdjustmentCPI']
df['SCE'] = df['SCE'] * df['AdjustmentCPI']*100
df['PG&E'] = df['PG&E'] * df['AdjustmentCPI']*100
df['SDG&E '] = df['SDG&E '] * df['AdjustmentCPI']*100
df['SMU06000003000000003($ per hour)'] = df['SMU06000003000000003($ per hour)'] * df['AdjustmentCPI']
df['STTMINWGCA($ per hour)'] = df['STTMINWGCA($ per hour)'] * df['AdjustmentCPI']
df['Real GDP per Employee($)'] = (df['CAMANRQGSP']*1000000)/(df['CAMFG(Thousands of ppl)']*1000)
df = df.round(2)
df = df.drop(columns=['AdjustmentCPI'])
df = df.rename(columns={
    'observation_date': 'Date',
    'FEDFUNDS(Percent': 'Federal Interest Rate (%)',
    'SMU06000002000000003($ per hour)': 'Construction Wage ($/hour)',
    'CACONSN(Thousands of ppl)': 'Construction Employees (Thousands)',
    'CACONSTRQGSP': 'Construction Real GDP (M$)',
    'SMU06000003000000003($ per hour)': 'Mfg Wage ($/hour)',
    'STTMINWGCA($ per hour)': 'Minimum Wage ($/hour)',
    'CAMANRQGSP': 'Real Mfg GDP (Millions of Dollars)',
    'CAMFG(Thousands of ppl)': 'Mfg Employees (Thousands)',
    'CAUR(percent)': 'Unemployment Rate (%)',
    'CASECCOMINVRGSP(Millions of Dollars)': 'Real Investment (M$)',
    'CACORPINCTX($)': 'Real Corporate Tax ($)',
    'SCE': 'SCE Real Energy Price ($/kWh)',
    'PG&E': 'PG&E Real Energy Price ($/kWh)',
    'SDG&E ': 'SDG&E Real Energy Price ($/kWh)',
    'PCUOMFGOMFG': 'Producer Price Index (USA)',
    'Average rate on all imports (%)': 'Average Tarrif Rate(%)',
})
df.to_csv('StateMfgData.csv', index=False)

